In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
import zipfile

# Define the zip file and destination folder
zip_path = r'../data/interim/mouse_p1p2_final.zip'  
extract_folder = "../data/interim/unzipped/"  

# Unzip the folder
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"Extracted to {extract_folder}")

# Define the zip file and destination folder
zip_path = r'../data/interim/weblog_p1p2.zip'  
extract_folder = "../data/interim/unzipped/"  

# Unzip the folder
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"Extracted to {extract_folder}")

# Unzipped data will be gitignored

Extracted to ../data/interim/unzipped/
Extracted to ../data/interim/unzipped/


In [16]:
# Load the data
file_path = "../data/interim/unzipped/mouse_p1p2_final.csv"
df = pd.read_csv(file_path)

# Drop duplicates, drop NAs, and sort values
# df = df.drop_duplicates(subset=['session_id', 'mousemove_times'], keep='first')
# df = df.sort_values(by=['session_id', 'mousemove_times']).reset_index(drop=True)
# df = df.dropna(subset=['session_id', 'mousemove_times', 'mousemove_type', 'mousemove_behaviour', 'label'])

# Extract the mousemove_x and mousemove_y coordinates from the 'mousemove_behaviour' string column
df[['mousemove_x', 'mousemove_y']] = df['mousemove_behaviour'].str.replace(r'[()]', '', regex=True).str.split(',', expand=True)

# Convert mousemove_x and mousemove_y to integer type
df['mousemove_x'] = df['mousemove_x'].astype(int)
df['mousemove_y'] = df['mousemove_y'].astype(int)

# Convert mousemove_type to categorical type
df = pd.get_dummies(df, columns=['mousemove_type'], dtype=int)

df

,session_id,mousemove_times,mousemove_behaviour,label,mousemove_x,mousemove_y,mousemove_type_cl,mousemove_type_cm,mousemove_type_cr,mousemove_type_m
0,0ht0u328t4mkgi01sp7mm07e01,447717248,"(0,5)",moderate_bot,0,5,0,0,0,1
1,0ht0u328t4mkgi01sp7mm07e01,447717265,"(0,6)",moderate_bot,0,6,0,0,0,1
2,0ht0u328t4mkgi01sp7mm07e01,447717281,"(0,7)",moderate_bot,0,7,0,0,0,1
3,0ht0u328t4mkgi01sp7mm07e01,447717298,"(0,8)",moderate_bot,0,8,0,0,0,1
4,0ht0u328t4mkgi01sp7mm07e01,447717315,"(0,9)",moderate_bot,0,9,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
3640494,rh5fn2njcglhb2avkv30id017u,1595505964115,"(1082,94)",human,1082,94,0,0,0,1
3640495,rh5fn2njcglhb2avkv30id017u,1595505964132,"(1140,54)",human,1140,54,0,0,0,1
3640496,rh5fn2njcglhb2avkv30id017u,1595505964149,"(1182,26)",human,1182,26,0,0,0,1
3640497,rh5fn2njcglhb2avkv30id017u,1595505964166,"(1214,8)",human,1214,8,0,0,0,1


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Select relevant columns for logistic classification
df_lc = df[['mousemove_x', 'mousemove_y', 'mousemove_type_cl', 'mousemove_type_cm', 'mousemove_type_cr', 'mousemove_type_m', 'label']]

# Features and target
X = df_lc.drop(columns='label')
y = df_lc['label']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit logistic regression model
clf = LogisticRegression(max_iter=1)
clf.fit(X_train, y_train)

# Predict class labels for test set
y_pred = clf.predict(X_test)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

# Optional: Detailed metrics for evaluation
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test Accuracy: 0.42413130064551574


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


Classification Report:
               precision    recall  f1-score   support

advanced_bot       0.42      1.00      0.60    308810
       human       0.00      0.00      0.00    232110
moderate_bot       0.00      0.00      0.00    187180

    accuracy                           0.42    728100
   macro avg       0.14      0.33      0.20    728100
weighted avg       0.18      0.42      0.25    728100

Confusion Matrix:
 [[308810      0      0]
 [232110      0      0]
 [187180      0      0]]
